In [29]:
from google.cloud import translate_v2 as translate
import requests
from bs4 import BeautifulSoup
from collections import Counter
import re
import pandas as pd
import os
from function_words import unique_function_words

181 164


In [30]:
print(os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/roneng100/API Keys/website-translation-419023-4572f2415643.json"

/Users/roneng100/API Keys/website-translation-419023-4572f2415643.json


In [31]:
def translate_text(text="Hello, world!", target_language="es"):

    translate_client = translate.Client()
    
    if isinstance(target_language, str):
        target_language = target_language.lower()
    
    result = translate_client.translate(text, target_language=target_language)
    
    print(u"Text: {}".format(result['input']))
    print(u"Translation: {}".format(result['translatedText']))
    print(u"Detected source language: {}".format(result['detectedSourceLanguage']))

    return result['translatedText']

if __name__ == "__main__":
    translate_text("Hello, world!", "es")

Text: Hello, world!
Translation: ¡Hola Mundo!
Detected source language: en


In [32]:
def extract_words_from_txt(file_path):
    # Try opening the file with different encoding if UTF-8 doesn't work
    try:
        # Try opening the file with the default UTF-8 encoding
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except UnicodeDecodeError:
        # If UTF-8 doesn't work, try with 'ISO-8859-1'
        with open(file_path, 'r', encoding='ISO-8859-1') as file:
            text = file.read()
    except Exception as e:
        # If other errors occur, raise them
        raise e
    
    # Split the text by whitespace to get the words
    words = text.split()
    
    return words

file_path = 'Subject_Terms.txt'
words = extract_words_from_txt(file_path)
print(len(words), words)

def normalize_words(word_list):
    # Use regular expression to remove any non-alphabetical characters (keeping only a-z and A-Z)
    normalized_words = [re.sub(r'[^a-zA-Z]', '', word) for word in word_list]
    # Remove any empty strings that may result from normalization
    normalized_words = [word for word in normalized_words if word]
    return normalized_words

# Example usage
test_words = ['(subject', 'fi?n@l', 'draft)', 'Agricul-tural', 'r3search', 'C0mmittee']
normalize_word = normalize_words(words)
print(len(normalize_word), normalize_word)
unique_words = set(normalize_word)
print(len(unique_words), unique_words)

1767 ['SUBJECT', 'TERMS:', 'FINAL', 'COMMITTEE', 'DRAFT', 'Agriculture', 'and', 'Food', 'Agricultural', 'marketing', 'Agricultural', 'practices', 'Agricultural', 'research', 'Agricultural', 'subsidies', 'Alcoholic', 'beverages', 'Aquaculture', 'Farmland', 'Food', 'assistance', 'Food', 'industry', 'Food', 'labeling', 'Food', 'safety', 'Genetically', 'modified', 'organisms', '(G.M.O.)', 'Livestock', 'Animals', 'Animal', 'diseases', 'Animal', 'protection', 'Service', 'animals', 'Veterinary', 'medicine', 'Wildlife', '{scope', 'note:', 'includes', 'tracking,', 'trafficking,', 'poaching,', 'and', 'refuges}', 'Arts', 'and', 'Culture', '{scope', 'note:', 'See', '�Intellectual', 'Property�', 'for', 'copyright-related', 'matters.}', 'Archives', 'Attire', 'Cultural', 'heritage', 'Libraries', 'Museums', 'Religious', 'buildings', 'Statues', 'and', 'monuments', 'Television', 'and', 'film', 'Commerce', '{scope', 'note:', 'Includes', 'e-commerce;', 'See', 'also', '�Foreign', 'Trade', 'and', 'Internati

In [33]:
filter_words = list(unique_words - unique_function_words)
print(len(filter_words), filter_words)

883 ['Infectious', 'passenger', 'service', 'Terrorism', 'Double', 'Gifted', 'Sector', 'parttime', 'Forfeiture', 'Income', 'Strategic', 'diet', 'fishing', 'commercial', 'Bridges', 'parks', 'Armed', 'products', 'review', 'industry', 'identify', 'guarantees', 'Labormanagement', 'Use', 'Maps', 'Childrens', 'stabilization', 'community', 'Recreation', 'Bicycling', 'jurisdictions', 'Floods', 'animals', 'Maintenance', 'loan', 'conditions', 'investment', 'Minority', 'Power', 'Libraries', 'prevention', 'Term', 'Refugees', 'News', 'Patients', 'Terminal', 'Juries', 'Territories', 'Trials', 'Nuclear', 'Weather', 'Guard', 'vehicles', 'organisms', 'Service', 'inheritance', 'Inheritance', 'Gambling', 'Blasphemy', 'crimes', 'Epidemics', 'participation', 'income', 'parties', 'cybercurrency', 'Disasters', 'Resources', 'DRAFT', 'biotechnology', 'information', 'restrictions', 'registers', 'Research', 'laws', 'Erosion', 'Animal', 'charitable', 'poaching', 'capital', 'Witnesses', 'buses', 'transmitted', 'Emp

In [34]:
def count_words_on_webpages_and_translate(urls, unique_words):
    website_word_frequencies = {}
    translator = translate.Client()

    for url in urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                text = soup.get_text().lower()

                # Translate the text from Spanish to English
                translated_text = translator.translate(text, target_language='en')['translatedText'].lower()
                # Clean the text
                clean_text = re.sub('[^a-z\s]', '', translated_text)
                word_counts = Counter(clean_text.split())
                print(word_counts)

                if url not in website_word_frequencies:
                    website_word_frequencies[url] = {}
                
                print(word_counts.items())
                
                word_frequencies = {word: count for word, count in word_counts.items() if word in unique_words}
                print(word_frequencies)
                website_word_frequencies[url] = word_frequencies
                print(word_frequencies.items())
                # all_word_frequencies
            else:
                print(f"Skipping URL due to non-200 status code: {url}, status code: {response.status_code}")
                website_word_frequencies[url] = response.status_code
        except requests.RequestException as e:
            print(f"Skipping URL due to request exception: {url}, exception: {e}")
    # print(website_word_frequencies)

    # Convert the list to a DataFrame and write to an Excel file
    df = pd.DataFrame(website_word_frequencies)
    df = df.T
    df.to_excel('word_frequencies_test.xlsx', index=False)

# Example usage:
urls = ['https://boe.es/boe/dias/2005/12/30/']
count_words_on_webpages_and_translate(urls, filter_words)

Counter({'of': 815, 'the': 702, 'and': 218, 'pdf': 187, 'kb': 186, 'other': 180, 'for': 177, 'formats': 173, 'page': 123, 'resolution': 123, 'boeb': 108, 'by': 99, 'which': 95, 'december': 89, 'boea': 77, 'to': 71, 'public': 63, 'pages': 63, 'ministry': 61, 'in': 57, 'is': 57, 'a': 54, 'community': 45, 'general': 37, 'award': 36, 'council': 35, 'autonomous': 33, 'file': 32, 'agreement': 30, 'state': 27, 'de': 25, 'on': 24, 'service': 22, 'city': 22, 'open': 22, 'contracting': 21, 'made': 20, 'regarding': 20, 'between': 20, 'project': 20, 'announcing': 20, 'tender': 20, 'announcement': 20, 'university': 19, 'research': 19, 'social': 18, 'order': 17, 'directorate': 17, 'call': 17, 'competition': 17, 'institute': 17, 'air': 17, 'works': 16, 'procedure': 16, 'provide': 16, 'services': 15, 'supply': 15, 'collaboration': 15, 'support': 15, 'maintenance': 14, 'administration': 13, 'justice': 13, 'catalonia': 13, 'madrid': 13, 'with': 13, 'as': 13, 'management': 13, 'command': 13, 'contract': 

In [45]:
import pandas as pd

# Example data
data = {
    "http://example.com": {"apple": 4, "banana": 2, "cherry": 1},
    "http://another.com": {"banana": 5, "date": 3},
    "http://site.com": {"apple": 1, "cherry": 2, "date": 2, "fig": 1},
}

# Prepare data for DataFrame
rows = []
for url, words in data.items():
    for word, count in words.items():
        rows.append((url, word, count))

# Create DataFrame
df = pd.DataFrame(rows, columns=["URL", "Word", "Count"])

# Export to Excel
with pd.ExcelWriter('output_with_merged_cells.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1', columns=["URL", "Word", "Count"])

    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Add the URL in the first column and merge as needed
    url_row_map = {}
    for i, (url, _) in enumerate(df.groupby('URL')):
        start_row = i * len(_) + 1  # +1 to account for header row
        end_row = start_row + len(_) - 1
        url_row_map[url] = (start_row, end_row)

        # Merge cells for the URL
        worksheet.merge_range(start_row, 0, end_row, 0, url)

    # Optionally adjust the column widths
    worksheet.set_column('A:A', 20)
    worksheet.set_column('B:B', 15)
    worksheet.set_column('C:C', 10)


In [36]:
import random
import string

def generate_word():
    """Generate a random word using lowercase letters."""
    return ''.join(random.choice(string.ascii_lowercase) for _ in range(random.randint(4, 10)))

def create_test_data(num_urls=5, words_per_url=120, error_url_index=2, error_message="HTTP Error 404: Not Found"):
    """Create a dictionary with specified number of URLs, each with a given number of unique words.
       One URL will be designated to return an error message instead of a word list.
    """
    data = {}
    for i in range(num_urls):
        url = f"http://example{i+1}.com"
        if i == error_url_index:
            # Assign an error message to this URL instead of word list
            data[url] = error_message
        else:
            words = {generate_word() for _ in range(words_per_url)}
            data[url] = list(words)
    return data

# Generate test data with one error URL
test_data = create_test_data()

# Display some of the data to verify
for url, content in test_data.items():
    print(f'{url}, {content}')

http://example1.com, ['edomlo', 'ttlspoyb', 'vbdbozut', 'hthnorjym', 'qlurl', 'sebpn', 'ygjwmeyq', 'avyfgn', 'ohsyq', 'upab', 'wdgiiu', 'yggzvw', 'ftrgyla', 'hfkc', 'zbelx', 'wcbtrpdw', 'fzlg', 'qoqccyybjw', 'mzcvmrnvr', 'nxukt', 'abuiilazlr', 'gogkkg', 'ofsjw', 'slibde', 'gukeud', 'odqerkxv', 'kzojynthvc', 'rdmudnqbky', 'tpetum', 'elqxiu', 'otzapojnu', 'slktaxo', 'vzpdhdv', 'rfbyvajo', 'ogzw', 'qkswyn', 'uzhybsfu', 'hfcnzgs', 'mvtqqy', 'bgrdiu', 'vjeorlw', 'oivqcoa', 'kzvnwb', 'covmnaeuya', 'ccrdtsi', 'egib', 'kowcjhu', 'qgvk', 'nfkbesbs', 'cpoqayuns', 'kqxxejpw', 'vhbukel', 'adptndpoud', 'vkhaq', 'kfijuck', 'htfgmkpe', 'lwlpopezq', 'yrphzpzhp', 'jinncytlmt', 'mtasaecb', 'kazsvbfl', 'jooclm', 'vkyxkcvrb', 'mhfxbjwyy', 'svqepbl', 'rzptl', 'wvrgtjbkz', 'patnomsnz', 'jwucrg', 'oszcy', 'fixvu', 'rank', 'cjsjdre', 'ouudx', 'stqoaozu', 'ilzhz', 'bxpjijn', 'dwgjurr', 'tatzgdmj', 'ovndwq', 'qtqpceaoq', 'lshcnykgt', 'gxdqcdwhw', 'ogkwcykbqx', 'bmmyr', 'koqgmx', 'cbelyt', 'albwpeel', 'qdly', 'b

In [44]:
import pandas as pd

# Example data structure
data = {
    "http://example.com": ["apple", "banana", "cherry"],
    "http://another.com": "HTTP Error 404: Not Found",  # This URL returned an error
    "http://site.com": ["apple", "cherry", "date", "fig"],
}

# Parameters
max_cols = 20  # Maximum number of columns for words

# Function to prepare data for DataFrame
def prepare_data(data, max_cols):
    rows = []
    merge_info = []
    for url, content in data.items():
        if isinstance(content, list):  # Check if the content is a list of words
            num_rows = -(-len(content) // max_cols)  # Ceiling division for rows
            merge_info.append((url, num_rows))

            # Split words into the appropriate number of rows
            for start in range(0, len(content), max_cols):
                end = start + max_cols
                row = [url] + content[start:end]
                row += [None] * (max_cols - len(content[start:end]))  # Correct padding calculation
                rows.append(row)
        else:
            # Handle error messages
            row = [url] + [content] + [None] * (max_cols - 1)  # Correct number of None to fill up the row
            rows.append(row)
            merge_info.append((url, 1))  # Only one row for error messages

    return rows, merge_info

# Prepare rows and merging information
data_rows, merge_info = prepare_data(test_data, max_cols)

# Create DataFrame
column_names = ['URL'] + [f"Word_{i+1}" for i in range(max_cols)]
df = pd.DataFrame(data_rows, columns=column_names)

# Export to Excel with merged cells and formatted headers
with pd.ExcelWriter('output_with_merged_cells_MATRIX.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Formatting for headers and cells
    header_format = workbook.add_format({'bold': True, 'align': 'left', 'valign': 'top'})
    text_format = workbook.add_format({'bold': True, 'align': 'center', 'valign': 'vcenter', 'font_color': 'black'})

    # Merge the word headers into one cell and name it "Unique Words"
    worksheet.merge_range(0, 1, 0, max_cols, 'Unique Words', header_format)

    # Apply formatting to URLs to prevent Excel from auto-formatting as hyperlinks
    for index, _ in enumerate(data_rows):
        worksheet.write_string(index + 1, 0, data_rows[index][0], text_format)  # +1 for header row

    # Apply merging for URL column based on the number of rows each URL occupies
    current_row = 1  # Excel rows start from 1 and account for the header
    for url, rows in merge_info:
        # Only apply the merging if more than one row is needed for the URL
        if rows > 1:
            worksheet.merge_range(current_row, 0, current_row + rows - 1, 0, url, text_format)
        current_row += rows

    # Set column widths
    worksheet.set_column(0, 0, 20)  # URL column wider
    for i in range(max_cols):
        worksheet.set_column(i+1, i+1, 15)  # Word columns